<a href="https://colab.research.google.com/github/shachimaurya23/Learning/blob/main/Tensorflow/Sarcasm__Prediction_using__tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [6]:
sentences=["i love my dog","I, love my cat","you love my dog!","dogs loves cat"]

In [9]:
tokenizer=Tokenizer(num_words=100,oov_token="<OOV") #num_words tells the max number of words allowed in the vocab  priority= freq.of words, sequesnce   eg 2 words occuring in sentences again and again then we see which comes first.
tokenizer.fit_on_texts(sentences)

In [10]:
word_index=tokenizer.word_index
print(word_index)

{'<OOV': 1, 'love': 2, 'my': 3, 'i': 4, 'dog': 5, 'cat': 6, 'you': 7, 'dogs': 8, 'loves': 9}


In [11]:
print(tokenizer.texts_to_sequences(["My hamster loves dogs and cat"]))

[[3, 1, 9, 8, 1, 6]]


In [ ]:
import tensorflow as tf
